In [1]:
import mlflow

In [2]:
# Step 2: Set up the MLflow tracking server
mlflow.set_tracking_uri("http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/")

In [3]:
# Set or create an experiment
mlflow.set_experiment("03 - feature_tuning_max_features: tfidf_bigram")

2025/11/20 16:52:29 INFO mlflow.tracking.fluent: Experiment with name '03 - feature_tuning_max_features: tfidf_bigram' does not exist. Creating a new experiment.


<Experiment: artifact_location='s3://sau-s3-mlflow-bucket/411462229921425448', creation_time=1763675549443, experiment_id='411462229921425448', last_update_time=1763675549443, lifecycle_stage='active', name='03 - feature_tuning_max_features: tfidf_bigram', tags={}>

In [4]:
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.ensemble import RandomForestClassifier
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report, confusion_matrix
import mlflow
import mlflow.sklearn
import matplotlib.pyplot as plt
import seaborn as sns
import pandas as pd
import os

In [5]:
df = pd.read_csv(
    '../../data/processed/reddit_clean_final.csv',
    keep_default_na=False,
    na_filter=False
)
df.shape

(36607, 2)

In [ ]:
# Step 1: Function to run the experiment
def run_experiment_tfidf_max_features(max_features):
    ngram_range = (1, 2)  # Bigram setting

    # Step 2: Vectorization using TF-IDF with varying max_features
    vectorizer = TfidfVectorizer(ngram_range=ngram_range, max_features=max_features)

    X_train, X_test, y_train, y_test = train_test_split(df['clean_comment'], df['category'], test_size=0.2, random_state=42, stratify=df['category'])

    X_train = vectorizer.fit_transform(X_train)
    X_test = vectorizer.transform(X_test)

    # Step 4: Define and train a Random Forest model
    with mlflow.start_run() as run:
        # Set tags for the experiment and run
        mlflow.set_tag("mlflow.runName", f"TFIDF_bigrams_max_features_{max_features}")
        mlflow.set_tag("experiment_type", "feature_engineering")
        mlflow.set_tag("model_type", "RandomForestClassifier")

        # Add a description
        mlflow.set_tag("description", f"RandomForest with TF-IDF bigrams, max_features={max_features}")

        # Log vectorizer parameters
        mlflow.log_param("vectorizer_type", "TF-IDF")
        mlflow.log_param("ngram_range", ngram_range)
        mlflow.log_param("vectorizer_max_features", max_features)

        # Log Random Forest parameters
        n_estimators = 200
        max_depth = 15

        mlflow.log_param("n_estimators", n_estimators)
        mlflow.log_param("max_depth", max_depth)

        # Initialize and train the model
        model = RandomForestClassifier(n_estimators=n_estimators, max_depth=max_depth, random_state=42)
        model.fit(X_train, y_train)

        # Step 5: Make predictions and log metrics
        y_pred = model.predict(X_test)

        # Log accuracy
        accuracy = accuracy_score(y_test, y_pred)
        mlflow.log_metric("accuracy", accuracy)

        # Log classification report
        classification_rep = classification_report(y_test, y_pred, output_dict=True)
        for label, metrics in classification_rep.items():
            if isinstance(metrics, dict):
                for metric, value in metrics.items():
                    mlflow.log_metric(f"{label}_{metric}", value)

        # Log confusion matrix
        conf_matrix = confusion_matrix(y_test, y_pred)
        plt.figure(figsize=(8, 6))
        sns.heatmap(conf_matrix, annot=True, fmt="d", cmap="Blues")
        plt.xlabel("Predicted")
        plt.ylabel("Actual")
        plt.title(f"Confusion Matrix: TF-IDF Bigrams, max_features={max_features}")
        plt.savefig("../../images/evaluation/03_confusion_matrix.png")
        mlflow.log_artifact("../../images/evaluation/03_confusion_matrix.png")
        plt.close()

        # Log the model
        mlflow.sklearn.log_model(model, f"random_forest_model_tfidf_bigrams_{max_features}")

# Step 6: Test various max_features values
max_features_values = [1000, 2000, 3000, 4000, 5000, 6000, 7000, 8000, 9000, 10000]

for max_features in max_features_values:
    run_experiment_tfidf_max_features(max_features)

d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2025/11/20 17:02:54 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 17:03:42 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_bigrams_max_features_1000 at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448/runs/f28945a5d5a54ab09dd8df04d25238bc
🧪 View experiment at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448


2025/11/20 17:04:38 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 17:04:50 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_bigrams_max_features_2000 at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448/runs/3529b1011330442f82eaffd00ed954bb
🧪 View experiment at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448


d:\SAURABH\spring 2024\MLOps Project\CreatorInsight AI Platform\.venv\lib\site-packages\boto3\compat.py:84: PythonDeprecationWarning: Boto3 will no longer support Python 3.9 starting April 29, 2026. To continue receiving service updates, bug fixes, and security updates please upgrade to Python 3.10 or later. More information can be found here: https://aws.amazon.com/blogs/developer/python-support-policy-updates-for-aws-sdks-and-tools/
  warnings.warn(warning, PythonDeprecationWarning)
2025/11/20 17:05:32 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 17:05:45 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


...

🏃 View run TFIDF_bigrams_max_features_8000 at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448/runs/12e0eb7a2764479c885ba86a99126212
🧪 View experiment at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448


2025/11/20 17:11:04 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 17:11:16 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_bigrams_max_features_9000 at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448/runs/5f3c5d99e00041178cec1f1b088270a7
🧪 View experiment at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448


2025/11/20 17:11:57 WARNING mlflow.models.model: `artifact_path` is deprecated. Please use `name` instead.
2025/11/20 17:12:12 WARNING mlflow.models.model: Model logged without a signature and input example. Please set `input_example` parameter when logging the model to auto infer the model signature.


🏃 View run TFIDF_bigrams_max_features_10000 at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448/runs/36230e308473404888ef607c4504a9f6
🧪 View experiment at: http://ec2-13-220-176-126.compute-1.amazonaws.com:5000/#/experiments/411462229921425448
